# Grover's Algorithm <br>
**1. Create a uniform Superposition of all the possible elements in which we are looking for** <br>
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\ket{\psi_{init.}}=\frac{1}{\sqrt{N}}\sum_{i=1}^{N}\ket{i}=\frac{1}{\sqrt{N}}(\ket{1}+\ket{2}+....+\ket{N})$

**2. Apply the oracle**  <br>
An oracle is an entity(eg., a function, operator or person) that knows the answer to the problem.<br>
-The oracle doesn't know the answer explicitly <br>
-Can only verify if a guess is right/wrong
here oracle will flip the sign of the solution state(s)

**3. Apply the amplifier(Grover operator)** <br>
to the flipped state(solution state)

### Practical example: The Inheritance <br>
We have an amount of property that we want to divide between two siblings. <br>
<code>property_prices = [4, 8, 6, 3, 12, 15]   #units in thousands of dollars</code>

In our case, we have 6 different properties. We want to see if we can partition them into two sets that have the same value. <br>

We will have 6 binary variables:<br>
x_i = 0 if the i-th property goes to the first sibling.<br>
x_i = 1 if the i-th property goes to the second sibling. <br>

Steps: <br>
1. We create a superposition of all possible asset allocations 
2. Through the oracle, we mark those distributions that satisy the request
3. We amplify the probability of observing solution states through Grover's operator

**Hadamard Gate** <br>
Used to create uniform superposition of computational basis. <br>
$H\ket{0}=\frac{1}{\sqrt{2}}[\ket{0}+\ket{1}]$ <br>
$H\ket{1}=\frac{1}{\sqrt{2}}[\ket{0}-\ket{1}]$ <br>
On the boch-sphere, it can be decomposed into two rotations: first apply rotation about the y-axis by $90\deg$ and then apply rotation about the x-axis by $180\deg$ <br>
Matrix representation: $ H = \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$ <br>

In [1]:
import pennylane as qml
from pennylane import numpy as np

property_prices = [4, 8, 6, 3, 12, 15] # 48 total

variables_wires = [0, 1, 2, 3, 4, 5]
aux_oracle_wires = [6, 7, 8, 9, 10, 11] 

In [2]:
def oracle(variables_wires, aux_oracle_wires):
    
    def add_k_fourier(k, wires):
        for j in range(len(wires)):
            qml.RZ(k * np.pi / (2**j), wires=wires[j])
    
    def value_second_sibling():
        qml.QFT(wires = aux_oracle_wires)  #convert the state from the computational basis into the Fourier basis by applying the QFT
        for wire in variables_wires:
            qml.ctrl(add_k_fourier, control=wire)(
                property_prices[wire],
                wires = aux_oracle_wires
            )
        value_second_sibling()
        qml.FlipSign(
            sum(property_prices) //2,
            wires = aux_oracle_wires
        )
        qml.adjoint(qml.QFT)(wires = aux_oracle_wires)  # return to computational basis
    
    value_second_sibling()
    qml.FlipSign(sum(property_prices) // 2, wires = aux_oracle_wires)
    qml.adjoint(value_second_sibling)()

In [3]:
dev = qml.device("default.qubit", wires = variables_wires + aux_oracle_wires)

@qml.qnode(dev)
def circuit():
    
    # step 1 
    for wire in variables_wires:
        qml.Hadamard(wires = wire)
        
    #step 2
    oracle(variables_wires, aux_oracle_wires)
    
    #step 3
    qml.GroverOperator(wires = variables_wires)
    
    return qml.probs(wires = variables_wires)

In [ ]:
import matplotlib.pyplot as plt

values = circuit()
plt.bar(range(len(values)), values)